In [1]:
from dataset_io import *
from rgb_channels import *
import os
import cv2
import numpy as np
import random as ra
from tqdm import tqdm
from profiler import *

## Gradient method

In [2]:
def include_border(rect):
    ((x1,y1),(x2,y2))=rect
    border=IMG_SAFETY_BORDER
    x1+=border
    x2+=border
    y1+=border
    y2+=border
    return ((x1,y1),(x2,y2))

def add_border(image):
    border=IMG_SAFETY_BORDER
    return cv2.copyMakeBorder(image, border, border, border, border, cv2.BORDER_REFLECT)

def update_optims(rect):
    ((x1,y1),(x2,y2))=rect
    w,h=(abs(x1-x2), abs(y1-y2))
    global w_max,h_max,w_min,h_min
    w_max=max(w,w_max)
    h_max=max(h,h_max)
    w_min=min(w,w_min)
    h_min=min(h,h_min) 
    
def extract_pallet_rectangles_from_image(rects, obj, filename, scene_name, channels_mode=False):
    imgs = [obj] if not channels_mode else obj #wrap image, don't wrap channels
    for j in range(len(rects)):
        rect=rects[j]
        if channels_mode:
            rect=include_border(rect)
            rect=correct_rect_ratio(rect)
            ((x1,y1),(x2,y2))=add_margin(rect)
            cropped_imgs = [img[y1:y2, x1:x2] for img in imgs]
            cropped_imgs = [cv2.resize(img, (120,24)) for img in cropped_imgs]
            save_image(cropped_imgs[0], scene_name, filename, "pallet_rectangles_channels",str(j)+"_ch1",True)
            save_image(cropped_imgs[1], scene_name, filename, "pallet_rectangles_channels",str(j)+"_ch2",True)
            global pallet_rectangles_channels
            pallet_rectangles_channels.append(cropped_imgs)
        else: 
            img = imgs[0] #unwrap image
            (x1,y1),(x2,y2)=rect
            cropped_img = img[y1:y2, x1:x2]
            h,w,_=cropped_img.shape
            hole_1=((int(0.126*w), int(0.3056*h)),(int(0.409*w), int(h)))
            hole_2=((int(0.5906*w), int(0.3056*h)),(int(0.875*w), int(h)))
            cv2.rectangle(cropped_img, hole_1[0], hole_1[1], (255,255,255, 0), -1)
            cv2.rectangle(cropped_img, hole_2[0], hole_2[1], (255,255,255, 0), -1)
            save_image(cropped_img, scene_name, filename, "pallet_rectangles_rgb", j)
            global pallet_rectangles
            pallet_rectangles_rgb.append(cropped_img)
        
def convert_to_channels(image):
    img = cv2.UMat(image)
    img = cv2.GaussianBlur(img, (3,3), sigmaX=0, sigmaY=0)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY )
    gradX = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3, scale=1 )
    gradY = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3, scale=1 )
    mag, ori = cv2.cartToPolar(gradX, gradY, angleInDegrees=True)

    ori_ranges = [(0,60), (60,120), (120,180), (180,240), (240,300), (300,360)]
    oris=[cv2.inRange(ori,l,r) for (l,r) in ori_ranges]
    mags= [cv2.copyTo(mag, ori_x) for ori_x in oris]
    channels=[gray,mag]+mags
    channels=[cv2.UMat.get(ch) for ch in channels]
    channels=np.moveaxis(channels, 0, -1)
    return channels

def convert_to_rgb_channels(image):
    channels=to_rgb_channels(image, "HSCMYb")
    return channels
    
def save_channels(channels, filename, scene_name, output_dir_name):
    ch1,ch2=np.array_split(channels, 2,2)
    save_image(ch1, scene_name, filename, output_dir_name,"ch1",True)
    save_image(ch2, scene_name, filename, output_dir_name,"ch2",True)

def extract_background_rectangles_from_image(rects, obj, filename, scene_name, channels_mode=False):
    imgs = [obj] if not channels_mode else obj #wrap image, don't wrap channels
    if rects:
        pallets_y1=99999
        pallets_y2=0
        sample_pallet_w=0
        y1s=[]
        y2s=[]
        hs=[]
        for j in range(len(rects)):
            rect=rects[j]
            rect=correct_rect_ratio(rect)
            ((x1,y1),(x2,y2))=add_margin(rect)
            w,h=(abs(x1-x2), abs(y1-y2))
            y1s.append(y1)
            y2s.append(y2)
            h=abs(y1-y2)
            hs.append(h)
        h=int(np.mean(hs))
        w=5*h
        y1=min(y1s)
        y2=max(y2s)
        upper_background = imgs[0][0:y1]
        lower_background = imgs[0][y2:]
        
        img_h,img_w,_=imgs[0].shape
        sampling_x1=0
        sampling_x2=img_w-w
        sampling_upper_y1=0
        sampling_upper_y2=y1-h
        sampling_lower_y1=y2
        sampling_lower_y2=img_h-h
        
        can_sample_upper=sampling_upper_y2>0
        can_sample_lower=sampling_lower_y2>sampling_lower_y1
        
        samples_count=4
        if (can_sample_upper or can_sample_lower) and sampling_x2>0: 
            for i in range(samples_count):
                sampled_x=ra.randint(sampling_x1, sampling_x2)
                sampling_done=False
                if i%2 and can_sample_upper:
                    sampled_y=ra.randint(sampling_upper_y1, sampling_upper_y2)
                    sampling_done=True
                elif can_sample_lower:
                    sampled_y=ra.randint(sampling_lower_y1, sampling_lower_y2)
                    sampling_done=True
                if sampling_done:
                    samples=[img[sampled_y:sampled_y+h, sampled_x:sampled_x+w] for img in imgs]
                    samples=[cv2.resize(sample, (120,24)) for sample in samples]
                    if channels_mode:
                        save_image(samples[0], scene_name, filename, "backgrounds_channels",str(i)+"_ch1",True)
                        save_image(samples[1], scene_name, filename, "backgrounds_channels",str(i)+"_ch2",True)
                        global background_rectangles_channels
                        background_rectangles_channels.append(samples)
                    else:
                        sample=samples[0] #unwrap image
                        save_image(sample, scene_name, filename, "backgrounds_rgb", i)
                        global background_rectangles
                        background_rectangles_rgb.append(sample)

### Highest level functions

In [3]:
def process_image_optims_extraction(filename, scene_name, label_resolution, rects):
    for rect in rects:
        ((x1,y1),(x2,y2))=rect
        rects=include_border(rect)
        rects=correct_rect_ratio(rects)
        update_optims(rects)

def process_image_channels_extraction(filename, scene_name, label_resolution, rects):
    image=imread_resized(scene_name, filename, label_resolution)
    channels=convert_to_channels(image)
    save_channels(channels, filename, scene_name, "channels")
    
def process_image_rgb_channels_extraction(filename, scene_name, label_resolution, rects):
    image=imread_resized(scene_name, filename, label_resolution)
    channels=convert_to_rgb_channels(image)
    save_channels(channels, filename, scene_name, "rgb_channels")
    
def process_image_structural_extraction(filename, scene_name, label_resolution, rects):
    channels=read_split_channels(scene_name, filename)
    channels_bordered=[add_border(ch) for ch in channels]
    extract_pallet_rectangles_from_image(rects, channels_bordered, filename, scene_name, True)
    extract_background_rectangles_from_image(rects, channels, filename, scene_name, True)
    
def process_image_rgb_extraction(filename, scene_name, label_resolution, rects):
    image=imread_resized(scene_name, filename, label_resolution)
    extract_pallet_rectangles_from_image(rects, image, filename, scene_name)
    extract_background_rectangles_from_image(rects, image, filename, scene_name)

def process_data(fun):
    [fun(*row) for row in tqdm(walk_dataset())]    

### Main

In [4]:
IMG_SAFETY_BORDER=130

w_max=h_max=0
w_min=h_min=99999

pallet_rectangles_rgb=[]
background_rectangles_rgb=[]
background_rectangles_channels=[]
pallet_rectangles_channels=[]

MODE_CHANNELS_EXTRACTION=process_image_channels_extraction
MODE_RGB_CHANNELS_EXTRACTION=process_image_rgb_channels_extraction
MODE_STRUCTURAL_EXTRACTION=process_image_structural_extraction
MODE_RGB_EXTRACTION=process_image_rgb_extraction
MODE_OPTIMS=process_image_optims_extraction
MODE=MODE_RGB_CHANNELS_EXTRACTION

In [ ]:
profiled('process_data(MODE)', globals(), locals())

 90%|█████████ | 1884/2092 [04:17<00:31,  6.64it/s]

In [ ]:
os.chdir(PROJECT_PATH) 

if MODE==MODE_STRUCTURAL_EXTRACTION:
    np.save("background_rectangles_channels",background_rectangles_channels)
    np.save("pallet_rectangles_channels",pallet_rectangles_channels)
    print(np.array(background_rectangles_channels).shape)
    print(np.array(pallet_rectangles_channels).shape)
elif MODE==MODE_RGB_EXTRACTION:
    np.save("background_rectangles_rgb",background_rectangles_rgb)
    np.save("pallet_rectangles_rgb",pallet_rectangles_rgb)
    print(np.array(background_rectangles_rgb).shape)
    print(np.array(background_rectangles_rgb).shape)
elif MODE==MODE_OPTIMS:
    print("w_max: ",w_max,"h_max: ",h_max)
    print("w_min: ",w_min,"h_min: ",h_min)